### Erstellung von Bi- und Trigrammen

beispielhaft werden die Daten geladen, tokenized und bigramme angereichert

https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/lda_training_tips.ipynb


In [3]:
import nltk
stopwords = nltk.corpus.stopwords.words('german')

In [4]:
tokSentences = []
for sentence in kuko:
    tok = [word for sent in nltk.sent_tokenize(sentence) for word in nltk.word_tokenize(sentence)]
    #print(tok)
    tok = [t.lower() for t in tok if len(t)>2] 
    tok = [w for w in tok if not w in stopwords]  
    #tok = [[token for token in doc if not token.isnumeric()] for doc in tok]
    tokSentences.append(tok)

In [5]:
import gensim
bigram = gensim.models.Phrases(tokSentences, min_count=20)
trigram = gensim.models.Phrases(bigram[tokSentences], min_count=20)

C:\Users\cb2lehk\AppData\Local\Continuum\Anaconda3_4\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\cb2lehk\AppData\Local\Continuum\Anaconda3_4\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [6]:
bigram.__dict__

{'delimiter': b'_',
 'max_vocab_size': 40000000,
 'min_count': 20,
 'min_reduce': 1,
 'progress_per': 10000,
 'threshold': 10.0,
 'vocab': defaultdict(int,
             {b'internet-marketing': 6,
              b'wohneinheiten_vergangenen': 7,
              b'dabei_\xc3\xbcberblick': 11,
              b'o.\xc3\xa4_geplant': 4,
              b'lediglich_wenigen': 1,
              b'rechnerisch_haushalts\xc3\xbcberschuss': 6,
              b'hauskauf_lebenspartner': 2,
              b'erfreut_warten': 3,
              b'angeboten.vorschlag': 1,
              b'krankenkasse_streit': 2,
              b'pentling_heute': 3,
              b'wohnanlage': 37,
              b'sollten.bei': 1,
              b'er\xc3\xb6ffnungsprotokoll_testaments': 3,
              b'itzehoe_beiden': 6,
              b'sicherheitsorienntiert_geldanlagen': 9,
              b'er\xc3\xb6ffnete_girokonto': 31,
              b'eur_usa': 2,
              b'lag_nv-bescheinigung': 6,
              b'wegen_nachfinanzierung

In [7]:

testList = []
testList.append("das ist ein separates konto ein euro ansehen".split(" "))

# bigramm-phrasen können aus neuen Sätzen (in Listen!) extrahiert werden  
for bigram, score in bigram.export_phrases(testList):
    print(u'{0}   {1}'.format(phrase, score))


In [8]:
text = []
for sent in tokSentences:
    text.append(trigram[sent]) # append the bigrams version of the text

C:\Users\cb2lehk\AppData\Local\Continuum\Anaconda3_4\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [9]:
# Remove rare and common tokens.

from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(text)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=5, no_above=0.5)

In [10]:
corpus = [dictionary.doc2bow(txt) for txt in text]
#print(corpus)

In [11]:
from gensim import corpora, models, similarities
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=50, id2word = dictionary, passes=20)

print(ldamodel.print_topics(num_topics=50, num_words=10))

[(0, '0.173*"thema" + 0.086*"besteht" + 0.044*"weiteres" + 0.043*"absicherung" + 0.041*"deutschen_bank" + 0.032*"mal" + 0.031*"rahmen" + 0.029*"familie" + 0.027*"desweiteren" + 0.024*"wissen"'), (1, '0.120*"überlegen" + 0.057*"kunde" + 0.055*"bestehenden" + 0.050*"jedoch" + 0.046*"müssen" + 0.037*"möchten" + 0.036*"aktuellen" + 0.030*"kredite" + 0.029*"angebote" + 0.026*"akt"'), (2, '0.202*"euro" + 0.071*"bleiben" + 0.055*"interesse" + 0.043*"entschieden" + 0.037*"vorerst" + 0.027*"informationen" + 0.023*"interessiert" + 0.019*"möchten" + 0.018*"wohnt" + 0.017*"finanzieren"'), (3, '0.095*"planen" + 0.053*"verkaufen" + 0.030*"februar" + 0.027*"zukunft" + 0.026*"investitionen" + 0.022*"premium" + 0.020*"kurz" + 0.018*"brauchen" + 0.017*"monatliches_sparen" + 0.015*"alte"'), (4, '0.102*"wegen" + 0.098*"gewünscht" + 0.073*"bekommen" + 0.055*"zwei" + 0.055*"bausparvertrag" + 0.052*"angaben" + 0.048*"verkauf" + 0.024*"davon" + 0.017*"bav" + 0.016*"umsätze"'), (5, '0.102*"onlinebanking" + 0.1

In [12]:
ldamodel.save('ldamodel')

In [13]:
import pyLDAvis.gensim

plot = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(plot)